# Gradients in Myia

We derive gradients for reverse mode using the technique described in [Reverse-Mode AD in a Functional Framework: Lambda the Ultimate Propagator](https://engineering.purdue.edu/~qobi/papers/toplas2008.pdf) by B. A. Pearlmutter and J. M. Siskind.

## Notation

We use a different notation in the code than in the paper:

| Paper | Myia (short) | Myia (code) | Note
|---|---|---|---
| $\overset{\leftharpoonup} x$ | $\blacktriangleright x$ | `fprop_x` | Compute values forward, mirroring the original computation. If $y = f(x)$ then ($\blacktriangleright y, \blacktriangleleft y) = \blacktriangleright f(\blacktriangleright x)$
| $\overline{x}$ | $\blacktriangleleft x$ | `bprop_x` | A function to propagate gradients backward. If $y = f(x)$ then $(\nabla f, \nabla x) \mathrel{+}= \blacktriangleleft y(\nabla y)$
| $\overset{\leftharpoondown} x$ | $\nabla x$ | `sens_x` | This is the gradient with respect to `x`.


### Primitives

We define three primitives for use with gradients:

| Paper | Myia | Note
|---|---|---
| $\overset{\leftarrow} {\mathcal{J}} x$ | `J(x)` | Transforms $x$ into $\blacktriangleright x$. This is a code transform if $x$ is a function, a no-op if $x$ is numeric.
| $\overset{\leftarrow} {\mathcal{J}}^{-1} x$ | `Jinv(x)` | Maps $\blacktriangleright x$ back to $x$.
| $\mathbf{0}(x)$ | `zeros_like(x)` | Creates a zero with the same shape as $x$ ($0$ for a scalar, a matrix of zeros for a matrix, etc.)

In addition to these, we define a data structure called an `Env` which is indexed by nodes:

| Operation | Note
|---|---
| `newenv` | An empty environment.
| `embed(node)` | A key corresponding to the given node (see it as a macro of sorts).
| `env_getitem(env, key, default)` | Get the value corresponding to a key, falling back to default. The type of the return value is uniquely determined by the key.
| `env_setitem(env, key, value)` | Set the value corresponding to a key. The value's type must match the key's.
| `env_add(env1, env2)` | Add two environments. Values corresponding to the same key are added structurally.

An `Env` serves to store a closure's sensitivities. 


## Interface

The code transform is performed by `myia.grad.J` (internal function, not part of the user-facing API). Given an input function $f$ it generates code for two functions, $\blacktriangleright f$ and $\blacktriangleleft f$. `J` only returns the former, however, because $\blacktriangleleft f$ is nested in $\blacktriangleright f$, and is returned by $\blacktriangleright f$ when it is called. 

The transform is accessible in user code as the primitive `J` (the one in `myia.prim.py_implementations`). It can be used as follows:

```python
jf = J(f)            # Transform the function (jf <=> ▶f)
jx = J(x)            # Transform the argument
jy, df = jf(jx)      # Apply the function (df <=> ◀f)
y = Jinv(jy)         # Untransform the result. y here is equal to f(x)
dy = 1               # This is ∇y, our starting point for the gradient (no need to transform)
dydf, dydx = df(dy)  # This is df(x)/dx (no need to untransform)
```

* Notice that `jf(jx)`, or `▶f(▶x)`, returns two values. The first represents the original output of the function, whereas the second is the backpropagator through which we can compute the gradient. All functions transformed by `J` return two values.
* Notice that `df(dy)`, or `◀f(∇y)`, also returns two values (but it may return more!) The first value is always the gradient with respect to the function itself, which we may call `∇f`. If `f` is a closure, `∇f` contains gradients with respect each of its free variables. Usually, you would ignore these, but they are crucial for the algorithm's proper operation. The other return value is the gradient with respect to its input (which you do want). If `f` had more arguments, more values would be returned. In general, if `f` takes `n` arguments, `◀f` takes one argument (always) and returns `n + 1` values.

Note that this is a low-level interface and we will implement wrappers around `J` to make this process easier.

## Explanation

### Generating $\blacktriangleright f$

There are essentially two rules to remember. The first transforms a function application, the second transforms a function definition:

| Original code | Transformed code
|---|---
| `y = f(x)` | `▶y, ◀y = ▶f(▶x)`
| `def inner(x): return expr` | `def ▶inner(▶x): return ▶{expr}, ◀{expr}`

These are implemented in `myia.grad.(FPropAppRemapper, FPropRemapper, BPropRemapper)`. These three remappers generate the nodes for `▶f(▶x)`, `▶y` and `◀y` respectively, linking them together as appropriate.

### Generating $\blacktriangleleft f$

This transform is a little more complicated. Algorithmically it is a bit different from what is in the paper, because we are using a graph representation which gives us easy access to all of a node's uses. In a nutshell, every time a variable $x$ is used in a graph, we need to generate a contribution to its gradient, $\nabla x$. Do note that there is one $\nabla x$ for every graph that uses $x$: whereas there is indeed always just one $\blacktriangleright x$ and just one $\blacktriangleleft x$, there may be more than one $\nabla x$ (but no more than one for each graph).

Here are the rules from the paper:

| Original code | Transformed code
|---|---
| Initialization | `∇x = zeros_like(x)`
| `y = f(x)` | `∇f, ∇x += ◀y(∇y)`
| `def inner(x): return expr` | `∇fv1, ∇fv2, ... += ∇inner` for `fv1, ...` the free variables of `inner`.

In the above, `∇inner` would be accumulated into every time there is a call to `inner` in the original code. If `inner` is returned, then `∇inner` would be the input to `◀f`. `∇inner` includes contributions to the gradients with respect to each free variable. Since some or all of these variables may be `f`'s own inputs, `f` needs to take these contributions into account when calculating the gradients with respect to its inputs. Given that `◀inner` packs them in a tuple, then `◀f` can deconstruct that tuple and recover contributions, assuming there is a standard order or format for free variables.

Myia's algorithm for this is adapted to its representation and type system, but it is not radically different. Nonetheless, here are the differences:

1. The algorithm accumulates all contributions to a node's sensitivity by following its uses, it is not generating a linear sequence of accumulations like above.
2. The initialization with `zeros_like(x)` is only done if `x` is unused, since it is unnecessary otherwise. This is trivial to do thanks to the strategy described  in 1.
3. Instead of packing a closure's sensitivities in a tuple, we pack them in an `Env` structure. The `Env` type is a singleton, which makes it a little easier for the type system to deal with, but it is nonetheless type safe because the keys used to index it embed the types of the values they map to.

The implementation is contained in `myia.grad.(BPropAppRemapper, SensRemapper)`, which respectively generate the nodes for `◀y(∇y)` and something like `∇x = sum(◀y(∇y)[idx] for y, idx in uses(x))`. `SensRemapper` is where most of the complexity of the whole algorithm is.
